In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread

import tensorflow as tf

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, Lambda, Reshape, multiply
from keras.layers import Activation, Dropout, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.applications import ResNet50
from keras.utils import to_categorical
from keras import backend as K
from keras.layers import Layer
from sklearn.model_selection import train_test_split

from keras.optimizers import RMSprop,SGD
from keras.regularizers import l2

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
'''physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)'''

"physical_devices = tf.config.experimental.list_physical_devices('GPU')\nif len(physical_devices) > 0:\n    tf.config.experimental.set_memory_growth(physical_devices[0], True)"

In [4]:
#!unzip /content/AML.zip -d /content/

In [5]:
#!unzip /content/CALL.zip -d /content/

In [6]:
#!pip install imgaug

In [7]:
# Ruta de la carpeta de descargas
# Directorio de descargas
download_dir = os.path.expanduser("AML/AML1")  # Ruta de la carpeta de descargas

# Subdirectorios dentro de la carpeta de descargas
#subdirs = ["M0", "M1", "M2", "M3", "M4"]
subdirs = ["all", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "sanos"]

# Listas para almacenar imágenes y recuentos
images = []
directories = []
dircount = []

# Tamaño deseado de las imágenes redimensionadas
target_height = 150 #108, 200
target_width = 250  #160, 300

print("leyendo imagenes de ",download_dir)

# Recorrer cada subdirectorio
for subdir in subdirs:
    subdir_path = os.path.join(download_dir, subdir)
    images_count = 0

    # Recorrer archivos en el subdirectorio
    for root, _, filenames in os.walk(subdir_path):
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                filepath = os.path.join(root, filename)

                # Leer y redimensionar la imagen
                image = imread(filepath)
                image_resized = resize(image, (target_height, target_width), anti_aliasing=True, preserve_range=True)
                images.append(image_resized)
                #images.append(image)

                images_count += 1

    # Almacenar recuento y directorio
    dircount.append(images_count)
    directories.append(subdir)

# Mostrar estadísticas finales
print('Directorios leídos:', len(directories))
print("Imágenes en cada directorio:", dircount)
print('Suma total de imágenes en subdirectorios:', sum(dircount))

# Convertir la lista de imágenes a un array de numpy
#images_array = np.array(images)
'''for i in range(len(images)):
  images[i] = np.array(images[i], dtype=np.uint8)'''
# Convertir las imágenes a RGB
'''for i in range(len(images)):
  images[i] = images[i][..., :3]
'''
type(images)

leyendo imagenes de  AML/AML1


Directorios leídos: 9
Imágenes en cada directorio: [360, 360, 360, 360, 360, 360, 360, 360, 360]
Suma total de imágenes en subdirectorios: 3240


list

In [8]:
# Directorio que contiene las imágenes
directorio_imagenes = os.path.expanduser("AML/AML1")
directorio_imagenes

'AML/AML1'

# Nueva sección

---



In [9]:
labels = []
indice = 0
for cantidad in dircount:
  for i in range(cantidad):
    labels.append(indice)
  indice = indice + 1
print("Cantidad de etiquetas creadas: ", len(labels))

Cantidad de etiquetas creadas:  3240


In [10]:
AMLI=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    AMLI.append(name[len(name)-1])
    indice=indice+1

0 all
1 M1
2 M2
3 M3
4 M4
5 M5
6 M6
7 M7
8 sanos


In [11]:
type(AMLI)

list

In [12]:
AMLI

['all', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'sanos']

In [13]:
#for img in images:
    #print(img.shape)

In [14]:
y = np.array(labels)
#X = images
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy

# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)
type(X)

Total number of outputs :  9
Output classes :  [0 1 2 3 4 5 6 7 8]


numpy.ndarray

In [15]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir listas a arrays NumPy
#X_train = np.array(X_train)
#X_test = np.array(X_test)

# Normalizamos como float32 para poder dividirlos entre 0 y 1
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Valores entre 0..1
X_train /= 255
X_test /= 255

print('Training data shape : ', X_train.shape, y_train.shape)
print('Testing data shape : ', X_test.shape, y_test.shape)

img_size = X[0].shape
print(img_size)

Training data shape :  (2592, 150, 250, 3) (2592,)
Testing data shape :  (648, 150, 250, 3) (648,)
(150, 250, 3)


In [16]:
from keras.utils import to_categorical

# Codificar las etiquetas
y_train_encoded = to_categorical(y_train, num_classes=nClasses)
y_test_encoded = to_categorical(y_test, num_classes=nClasses)

# Modelos


---



In [17]:
# Modelo SENet
def se_block(input_tensor, ratio=16):
    channel_count = input_tensor.shape[-1]

    # Global average pooling que preserva las dimensiones originales
    x = GlobalAveragePooling2D()(input_tensor)

    # Capas Dense para aprender los pesos de excitación
    x = Dense(channel_count // ratio, activation='relu')(x)
    x = Dense(channel_count, activation='sigmoid')(x)

    # Redefinir las dimensiones para poder aplicar capas Dense
    #x = Reshape((1, 1, channel_count))(x)

    # Multiplicar el tensor de entrada por los pesos aprendidos
    x = multiply([input_tensor, x])
    return x

def senet_model(input_shape=img_size, ratio=16):
    input_tensor = Input(shape=input_shape)

    x = Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu')(input_tensor)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)

    x = Conv2D(256, (5, 5), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(x)
    x = se_block(x, ratio=ratio)

    #x = AveragePooling2D(pool_size=(2, 2))(x)
    # Aplanar el tensor antes de la capa Dense
    x = Flatten()(x)

    x = Dense(1024, activation='relu')(x)  # Agregada una capa densa
    #x = BatchNormalization()(x)
    x = Dropout(0.1)(x)  # Agregada capa de dropout

    output_tensor = Dense(nClasses, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

senet_model = senet_model(input_shape=img_size)

In [18]:
epochs1 = 31
batch_size1 = 32
train_size1 =2592

initial_learning_rate1 = 0.0001
final_learning_rate1 = 0.00001
learning_rate_decay_factor1 = (final_learning_rate1 / initial_learning_rate1)**(1/epochs1)
steps_per_epoch1 = int(train_size1/batch_size1)

lr_schedule1 = tf.keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=initial_learning_rate1,
                decay_steps=steps_per_epoch1,
                decay_rate=learning_rate_decay_factor1,
                staircase=True)
# Define optimizer using the learning rate schedule
optimizer1 = RMSprop(learning_rate=lr_schedule1)

In [19]:
senet_model.compile(optimizer=optimizer1, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
senet_model.fit(X_train, y_train_encoded, epochs=epochs1, batch_size=batch_size1, validation_data=(X_test, y_test_encoded))

Epoch 1/31
81/81 [==============================] - 10s 87ms/step - loss: 2.0299 - accuracy: 0.2079 - val_loss: 1.7751 - val_accuracy: 0.3441
Epoch 2/31
81/81 [==============================] - 5s 58ms/step - loss: 1.7544 - accuracy: 0.3391 - val_loss: 1.8682 - val_accuracy: 0.3117
Epoch 3/31
81/81 [==============================] - 5s 58ms/step - loss: 1.5919 - accuracy: 0.4244 - val_loss: 1.4560 - val_accuracy: 0.4599
Epoch 4/31
81/81 [==============================] - 5s 58ms/step - loss: 1.4429 - accuracy: 0.4776 - val_loss: 1.3718 - val_accuracy: 0.4738
Epoch 5/31
81/81 [==============================] - 5s 58ms/step - loss: 1.2949 - accuracy: 0.5382 - val_loss: 1.3163 - val_accuracy: 0.5077
Epoch 6/31
81/81 [==============================] - 5s 58ms/step - loss: 1.1679 - accuracy: 0.5876 - val_loss: 1.1622 - val_accuracy: 0.5818
Epoch 7/31
81/81 [==============================] - 5s 57ms/step - loss: 1.0383 - accuracy: 0.6343 - val_loss: 1.0452 - val_accuracy: 0.6466
Epoch 8/31
8

In [ ]:
senet_model.save('senet_model.h5')